In [45]:
import pandas as pd
import numpy as np
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows as df_to_row

In [36]:
from szp_funcs import load_month, print_df, months_year, custom_create_res, re_szp, bad_satus, gku_inn, get_fio, get_otv, months_year_rus

In [3]:
months = months_year[:6]

In [4]:
res = custom_create_res(months, load_month)

In [28]:
def szp_2(row, months):
    active_months = 0
    fot = 0
    for month in months:
        if pd.isna(row['sum_' + month]) == False:
            fot += row['sum_' + month]
            active_months += 1
    if active_months == 0:
        return np.nan
    return fot / active_months

In [29]:
prep = res[(res.inn.isin(gku_inn)) & (res['job_' + months[-1]].isna() == False) & (res['sum_' + months[-1]].isna() == False)]
prep['szp'] = prep.apply(lambda row: re_szp(row, months), axis=1)
prep['szp_2'] = prep.apply(lambda row: szp_2(row, months), axis=1)
#prep = prep[prep.szp.isna() == False]

C:\Users\PetukhovMD\AppData\Local\Temp\ipykernel_18176\223631930.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prep['szp'] = prep.apply(lambda row: re_szp(row, months), axis=1)
C:\Users\PetukhovMD\AppData\Local\Temp\ipykernel_18176\223631930.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prep['szp_2'] = prep.apply(lambda row: szp_2(row, months), axis=1)


In [39]:
static_names = {'inn': (np.nan, 'ИНН'), 'snils': (np.nan, 'СНИЛС'), 'szp': (np.nan, 'СЗП'), 'szp_teach': (np.nan, 'СЗП учителей'),
'szp_ped': (np.nan, 'СЗП педы'), 'fot': (np.nan, 'ФОТ'), 'fot_aup': (np.nan, 'ФОТ АУП'), 'top10_all': (np.nan, 'max 10% от всех работников'),
'bot10_all': (np.nan, 'min 10% от всех работников'), 'top10_teach': (np.nan, 'max 10% учителей'), 'bot10_teach': (np.nan, 'min 10% учителей'),
'bot50_teach': (np.nan, 'min 50% учителей'), 'bot50_ped': (np.nan, 'min 50% педагогов'), 'fot_isp': (np.nan, 'ФОТ НОУВП')}
per_month_names = {'sum': (np.nan, 'Начисления'), 'job': (np.nan, 'Должность'), 'day': (np.nan, 'Отработанный месяц'),
'type': (np.nan, 'Вид занятости'), 'stv': (np.nan, 'Ставка'), 'status': (np.nan, 'Статус на конец месяца'), 'status_pref': (np.nan, 'Статус на начало месяца'),
'wday': (np.nan, 'Отработано дней'), 'nday': (np.nan, 'Всего рабочих дней в месяце'), 'sum_all': (np.nan, 'Начисления всего'), 'spec': (np.nan, 'Направление деятельности')}
monthly_names = {key: value for key, value in zip([key + '_' + month for key in per_month_names.keys() for month in months_year], [(month, value[1]) for value in per_month_names.values() for month in months_year_rus])}
q_names = {key: value for key, value in zip([key + '_q_' + str(q) for key in static_names.keys() for q in range(1, 5)], [('кв_' + str(q), value[1]) for value in static_names.values() for q in range(1, 5)])}
translate = static_names | per_month_names | monthly_names | q_names

In [40]:
def print_with_trans(df_in, file, trans):
    df = df_in.copy()
    df.columns = pd.MultiIndex.from_tuples(df.rename(columns=trans).columns.to_list())
    row = [[] for i in df.columns.names]
    wb = Workbook()
    ws = wb.active
    for i in df.columns:
        for j in range(len(row)):
            row[j].append(i[j])
    for i in row:
        ws.append(i)
    for i in df_to_row(df, header=False, index=False):
        ws.append(i)
    wb.save(file + '.xlsx')

In [41]:
prep = prep.drop([j + i for i in months for j in ['stv_', 'type_', 'status_', 'status_pref_', 'spec_', 'day_']], axis=1)

In [42]:
prep

,inn,snils,sum_jan,sum_all_jan,wday_jan,nday_jan,job_jan,sum_feb,sum_all_feb,wday_feb,...,wday_may,nday_may,job_may,sum_jun,sum_all_jun,wday_jun,nday_jun,job_jun,szp,szp_2
8234,7702061938,001-437-035 85,109027.0,109027.0,16.0,16.0,Экскурсовод,109027.0,109027.0,19.0,...,18.0,18.0,Экскурсовод,93451.71,93451.71,18.0,21.0,Экскурсовод,109027.0,96847.356667
8235,7702061938,001-545-288 06,171550.0,171550.0,16.0,16.0,Начальник отдела,266550.0,266550.0,19.0,...,18.0,18.0,Начальник отдела,230550.00,230550.00,21.0,21.0,Начальник отдела,227550.0,213184.851667
8236,7702061938,001-550-908 97,80336.0,80336.0,16.0,16.0,Методист,80336.0,80336.0,19.0,...,18.0,18.0,Методист,80336.00,191036.52,21.0,21.0,Методист,95336.0,95336.000000
8237,7702061938,001-578-582 36,94678.0,94678.0,16.0,16.0,Уборщик территории,94678.0,94678.0,19.0,...,18.0,18.0,Уборщик территории,31259.90,78016.68,11.0,21.0,Уборщик территории,85678.0,76608.316667
8238,7702061938,002-620-361 82,48201.6,48201.6,16.0,16.0,Хормейстер,48201.6,48201.6,19.0,...,18.0,18.0,Хормейстер,48201.60,94763.92,21.0,21.0,Хормейстер,NaN,45280.291667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171488,9718071371,144-210-752 18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,89736.81,89736.81,17.0,17.0,Бухгалтер,NaN,89736.810000
171489,9718071371,145-941-023 60,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,30000.00,30000.00,NaN,NaN,Учитель,NaN,30000.000000
171490,9718071371,163-839-219 93,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,76276.81,76276.81,17.0,17.0,Бухгалтер,NaN,76276.810000
171491,9718071371,168-147-261 85,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,30000.00,30000.00,NaN,NaN,Учитель,NaN,30000.000000


In [22]:
fio = get_fio()
fio.columns

Index(['snils', 'fio'], dtype='object')

In [23]:
otv = get_otv()
otv.columns

Index(['inn', 'okr', 'otv', 'sch_name', 'ekis'], dtype='object')

In [47]:
ans = pd.merge(prep, fio, how='left')
ans = pd.merge(ans, otv[['inn', 'sch_name']])

In [48]:
ans

,inn,snils,sum_jan,sum_all_jan,wday_jan,nday_jan,job_jan,sum_feb,sum_all_feb,wday_feb,...,job_may,sum_jun,sum_all_jun,wday_jun,nday_jun,job_jun,szp,szp_2,fio,sch_name
0,7702061938,001-437-035 85,109027.0,109027.0,16.0,16.0,Экскурсовод,109027.0,109027.0,19.0,...,Экскурсовод,93451.71,93451.71,18.0,21.0,Экскурсовод,109027.0,96847.356667,Михайлова Юлия Ивановна,ГБУ МГДУ
1,7702061938,001-545-288 06,171550.0,171550.0,16.0,16.0,Начальник отдела,266550.0,266550.0,19.0,...,Начальник отдела,230550.00,230550.00,21.0,21.0,Начальник отдела,227550.0,213184.851667,Радзиминская Ирина Вячеславовна,ГБУ МГДУ
2,7702061938,001-550-908 97,80336.0,80336.0,16.0,16.0,Методист,80336.0,80336.0,19.0,...,Методист,80336.00,191036.52,21.0,21.0,Методист,95336.0,95336.000000,Черниченко Лариса Александровна,ГБУ МГДУ
3,7702061938,001-578-582 36,94678.0,94678.0,16.0,16.0,Уборщик территории,94678.0,94678.0,19.0,...,Уборщик территории,31259.90,78016.68,11.0,21.0,Уборщик территории,85678.0,76608.316667,Петрухин Валерий Анатольевич,ГБУ МГДУ
4,7702061938,002-620-361 82,48201.6,48201.6,16.0,16.0,Хормейстер,48201.6,48201.6,19.0,...,Хормейстер,48201.60,94763.92,21.0,21.0,Хормейстер,NaN,45280.291667,Кресман Татьяна Валентиновна,ГБУ МГДУ
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5304,9718071371,144-210-752 18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,89736.81,89736.81,17.0,17.0,Бухгалтер,NaN,89736.810000,Стенникова Людмила Викторовна,ГКУ ЦФО ДОНМ
5305,9718071371,145-941-023 60,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,30000.00,30000.00,NaN,NaN,Учитель,NaN,30000.000000,Кадерова Альфия Алиевна,ГКУ ЦФО ДОНМ
5306,9718071371,163-839-219 93,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,76276.81,76276.81,17.0,17.0,Бухгалтер,NaN,76276.810000,Копылова Анна Николаевна,ГКУ ЦФО ДОНМ
5307,9718071371,168-147-261 85,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,30000.00,30000.00,NaN,NaN,Учитель,NaN,30000.000000,Фесан Елена Юрьевна,ГКУ ЦФО ДОНМ


In [52]:
grouped_ans = ans.groupby('inn')
true_ans = 0
for group in grouped_ans.groups:
    if type(true_ans) == int:
        true_ans = grouped_ans.get_group(group).sort_values('szp', ascending=False).head(30)
        continue
    true_ans = pd.concat([true_ans, grouped_ans.get_group(group).sort_values('szp', ascending=False).head(30)])

In [53]:
true_ans

,inn,snils,sum_jan,sum_all_jan,wday_jan,nday_jan,job_jan,sum_feb,sum_all_feb,wday_feb,...,job_may,sum_jun,sum_all_jun,wday_jun,nday_jun,job_jun,szp,szp_2,fio,sch_name
10,7702061938,018-315-488 42,551269.0,551269.0,16.0,16.0,Директор,551269.00,551269.00,19.0,...,Директор,581269.00,1239611.16,21.0,21.0,Директор,557269.000000,599549.875000,Минько Нина Григорьевна,ГБУ МГДУ
1,7702061938,001-545-288 06,171550.0,171550.0,16.0,16.0,Начальник отдела,266550.00,266550.00,19.0,...,Начальник отдела,230550.00,230550.00,21.0,21.0,Начальник отдела,227550.000000,213184.851667,Радзиминская Ирина Вячеславовна,ГБУ МГДУ
26,7702061938,079-398-498 50,219867.0,219867.0,16.0,16.0,Руководитель структурного подразделения,264867.00,264867.00,19.0,...,Руководитель структурного подразделения,257038.31,281225.33,12.0,21.0,Руководитель структурного подразделения,226117.000000,236690.948333,Локотников Михаил Юрьевич,ГБУ МГДУ
29,7702061938,112-424-231 94,187886.0,187886.0,16.0,16.0,Начальник отдела,187886.00,187886.00,19.0,...,Начальник отдела,145532.19,145532.19,16.0,21.0,Начальник отдела,222886.000000,179453.975000,Коваленко Борис Владимирович,ГБУ МГДУ
21,7702061938,047-343-592 65,237550.0,237550.0,16.0,16.0,Начальник отдела,222550.00,222550.00,19.0,...,Начальник отдела,177550.00,335342.11,21.0,21.0,Начальник отдела,203383.333333,203383.333333,Александрова Светлана Станиславовна,ГБУ МГДУ
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4809,9718071371,115-610-335 12,254871.0,254871.0,16.0,16.0,Начальник отдела,241350.57,265129.26,14.0,...,Начальник отдела,304871.00,304871.00,21.0,21.0,Начальник отдела,298204.333333,273877.095000,Неврюева Екатерина Николаевна,ГКУ ЦФО ДОНМ
4892,9718071371,127-349-253 66,280087.0,280087.0,16.0,16.0,Начальник отдела,280087.00,280087.00,19.0,...,Начальник отдела,228712.10,228712.10,16.0,21.0,Начальник отдела,297479.000000,298610.380000,Майильян Елена Александровна,ГКУ ЦФО ДОНМ
4708,9718071371,079-668-729 49,299871.0,299871.0,16.0,16.0,Начальник отдела,294871.00,294871.00,19.0,...,Начальник отдела,244175.86,402765.06,18.0,21.0,Начальник отдела,296537.666667,325808.693333,Юрасова Виктория Владимировна,ГКУ ЦФО ДОНМ
4709,9718071371,079-767-751 49,279871.0,279871.0,16.0,16.0,Начальник отдела,274871.00,274871.00,19.0,...,Начальник отдела,289871.00,289871.00,21.0,21.0,Начальник отдела,295871.000000,308755.566667,Билле Дмитрий Викторович,ГКУ ЦФО ДОНМ


In [54]:
print_with_trans(true_ans, 'Списки сзп ЦГУ по ' + months[-1] + ' trans', translate)

In [27]:
print_df(ans, 'Списки сзп ЦГУ по ' + months[-1])